## 1.- Imports, setup and configure
### 1.1.- Imports
Bring in the different dependencies from installed standard modules

In [1]:
import sys
import time
import glob
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
from scipy import signal

Now the ad-hoc created modules for this project. We use the jupyter magics %load_ext autoreload and %autoreload set to 2. Imported classes are located in the ../scripts folder of our volume

In [2]:
import sys
sys.path.insert(0, '../../scripts')
# load the autoreload extension
%load_ext autoreload
# Set extension to reload modules every time before executing code
%autoreload 2

from video_asset_processor import video_asset_processor

### 1.2.- Custom functions
Add the necessary custom functions for the notebook

In [3]:
def read_metric_log(path, metric):
    if metric == 'vmaf':
        with open(path) as f:
            for line in f:
                if '= ' in line:
                    return float(line.split('= ')[-1])
    if metric == 'ms-ssim':
        ms_ssim_df = pd.read_csv(path)
        return(ms_ssim_df['ms-ssim'].mean())

### 1.3.- Configure the inputs
Setup the needed parameters to pass to the functions

In [4]:
# Enumerate the list of metrics to extract
# -hash_euclidean
# -hash_cosine
# -hash_hamming
# -temporal_difference (this creates two output columns): 
#  -temporal_difference_euclidean 
#  -temporal_difference_cosine

metrics_list = ['temporal_difference', 'temporal_canny', 'temporal_histogram_distance', 'temporal_cross_correlation', 'temporal_dct']

renditions_folders = [
'1080p',
'1080p_watermark',
'1080p_flip_vertical',
'1080p_rotate_90_clockwise',
'1080p_vignette',
'1080p_black_and_white',
'1080p_low_bitrate_4',
'720p',
'720p_vignette',
'720p_black_and_white',
'720p_low_bitrate_4',
'720p_watermark',
'720p_flip_vertical',
'720p_rotate_90_clockwise',
'480p',
'480p_watermark',
'480p_vignette',
'480p_black_and_white',
'480p_low_bitrate_4',
'480p_flip_vertical',
'480p_rotate_90_clockwise',
'360p',
'360p_watermark',
'360p_vignette',
'360p_black_and_white',
'360p_low_bitrate_4',
'360p_flip_vertical',
'360p_rotate_90_clockwise',
'240p',
'240p_watermark',
'240p_vignette',
'240p_black_and_white',
'240p_low_bitrate_4',
'240p_flip_vertical',
'240p_rotate_90_clockwise',
'144p',
'144p_watermark',
'144p_vignette',
'144p_black_and_white',
'144p_low_bitrate_4',
'144p_flip_vertical',
'144p_rotate_90_clockwise',
]
originals_path = '../../data/{}/'

## 2.- Iterate all assets in the data set and extract their metrics

In [ ]:
metrics_dict = {}
list = os.listdir(originals_path.format('1080p')) # dir is your directory path
number_assets = len(list)
print ('Number of assets: {}'.format(number_assets))
count = 0

for original_asset in glob.iglob(originals_path.format('1080p') + '**', recursive=False):
    count += 1
    if os.path.isfile(original_asset): # filter dirs
        print('Processing asset {} of {}: {}'.format(count, number_assets, original_asset))
        start_time = time.time()
        renditions_list = []

        for folder in renditions_folders:
            rendition_folder = originals_path.format(folder)
            renditions_list.append(rendition_folder + os.path.basename(original_asset))

        asset_processor = video_asset_processor(original_asset, renditions_list, metrics_list,4)

        asset_metrics_dict = asset_processor.process()

        dict_of_df = {k: pd.DataFrame(v) for k,v in asset_metrics_dict.items()}

        metrics_df = pd.concat(dict_of_df, axis=1).transpose().reset_index(inplace=False)
        metrics_df = metrics_df.rename(index=str, columns={"level_1": "frame_num", "level_0": "path"})
        
        renditions_dict = {}
        for rendition in renditions_list:
            rendition_dict = {}
            for metric in metrics_list:

                original_df = metrics_df[metrics_df['path']==original_asset][metric]
                original_df = original_df.reset_index(drop=True).transpose().dropna().astype(float)

                rendition_df = metrics_df[metrics_df['path']==rendition][metric]
                rendition_df = rendition_df.reset_index(drop=True).transpose().dropna().astype(float)

                if  'temporal' in metric:
                    x_original = np.array(original_df[rendition_df.index].values)
                    x_rendition = np.array(rendition_df.values)
  
                    [[manhattan]] = 1/abs(1-distance.cdist(x_original.reshape(1,-1), x_rendition.reshape(1,-1), metric='cityblock'))

                    rendition_dict['{}-euclidean'.format(metric)] = distance.euclidean(x_original, x_rendition)
                    rendition_dict['{}-manhattan'.format(metric)] = manhattan
                    rendition_dict['{}-cosine'.format(metric)] = distance.cosine(x_original, x_rendition)
                    rendition_dict['{}-cross-correlation'.format(metric)] = np.correlate(x_original, x_rendition)
                    rendition_dict['{}-mean'.format(metric)] = np.mean(x_rendition)
                    rendition_dict['{}-max'.format(metric)] = np.max(x_rendition)
                    rendition_dict['{}-std'.format(metric)] = np.std(x_rendition)
                    rendition_dict['{}-series'.format(metric)] = x_rendition
                else:
                    rendition_dict[metric] = rendition_df.mean()
                rendition_dict['size'] = os.path.getsize(rendition)
            renditions_dict[rendition] = rendition_dict

        metrics_dict[original_asset] = renditions_dict   

        elapsed_time = time.time() - start_time 
        print('Elapsed time:', elapsed_time)
        print('***************************')


Number of assets: 139
Processing asset 1 of 139: ../../data/1080p/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efea6f78650> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.6338694095611572
Converting <VideoCapture 0x7efea6f78650> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.6046168804168701
Comparing ../../data/1080p/VZpNs8Ob5XU.mp4
../../data/1080p/VZpNs8Ob5XU.mp4 Comparison elapsed time: 0.6026270389556885
PATH: /home/jovyan/work/data/1080p/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efea6f78650> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.6440639495849609
Comparing ../../data/1080p_watermark/VZpNs8Ob5XU.mp4
../../data/1080p_watermark/VZpNs8Ob5XU.mp4 Comparison elapsed time: 0.5931503772735596
PATH: /home/jovyan/work/data/1080p_watermark/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efea6f78650> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.68460083007

../../data/360p_vignette/VZpNs8Ob5XU.mp4 Comparison elapsed time: 0.6796793937683105
PATH: /home/jovyan/work/data/360p_vignette/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efe7d023ed0> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.13010501861572266
Comparing ../../data/360p_black_and_white/VZpNs8Ob5XU.mp4
../../data/360p_black_and_white/VZpNs8Ob5XU.mp4 Comparison elapsed time: 0.685239315032959
PATH: /home/jovyan/work/data/360p_black_and_white/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efe7d023e70> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.12108087539672852
Comparing ../../data/360p_low_bitrate_4/VZpNs8Ob5XU.mp4
../../data/360p_low_bitrate_4/VZpNs8Ob5XU.mp4 Comparison elapsed time: 0.656923770904541
PATH: /home/jovyan/work/data/360p_low_bitrate_4/VZpNs8Ob5XU.mp4
Converting <VideoCapture 0x7efe7d023cb0> to numpy arrays
Captured 4.033333333333333 seconds (121 frames)
Elapsed time: 0.12789106369018555
Comparing ../..

/opt/conda/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Elapsed time: 41.508394718170166
***************************
Processing asset 2 of 139: ../../data/1080p/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efe796067b0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.7375576496124268
Converting <VideoCapture 0x7efea6f78630> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.749537467956543
Comparing ../../data/1080p/ckO1MEL4caU.mp4
../../data/1080p/ckO1MEL4caU.mp4 Comparison elapsed time: 0.7681872844696045
PATH: /home/jovyan/work/data/1080p/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efe796065b0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.7031998634338379
Comparing ../../data/1080p_watermark/ckO1MEL4caU.mp4
../../data/1080p_watermark/ckO1MEL4caU.mp4 Comparison elapsed time: 0.777475118637085
PATH: /home/jovyan/work/data/1080p_watermark/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efea6f78630> to numpy arrays
Captured 4.0344827586206895 seconds

../../data/360p_vignette/ckO1MEL4caU.mp4 Comparison elapsed time: 0.7151584625244141
PATH: /home/jovyan/work/data/360p_vignette/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efe79606df0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.11322951316833496
Comparing ../../data/360p_black_and_white/ckO1MEL4caU.mp4
../../data/360p_black_and_white/ckO1MEL4caU.mp4 Comparison elapsed time: 0.7051606178283691
PATH: /home/jovyan/work/data/360p_black_and_white/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efe796062f0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.11016416549682617
Comparing ../../data/360p_low_bitrate_4/ckO1MEL4caU.mp4
../../data/360p_low_bitrate_4/ckO1MEL4caU.mp4 Comparison elapsed time: 0.7430269718170166
PATH: /home/jovyan/work/data/360p_low_bitrate_4/ckO1MEL4caU.mp4
Converting <VideoCapture 0x7efea6f78630> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.1255033016204834
Comparing .

...

../../data/360p_rotate_90_clockwise/4sCVouieJUc.mp4 Comparison elapsed time: 0.6818702220916748
PATH: /home/jovyan/work/data/360p_rotate_90_clockwise/4sCVouieJUc.mp4
Converting <VideoCapture 0x7efe7c4dedb0> to numpy arrays
Captured 4.04 seconds (101 frames)
Elapsed time: 0.06868195533752441
Comparing ../../data/240p/4sCVouieJUc.mp4
../../data/240p/4sCVouieJUc.mp4 Comparison elapsed time: 0.7049736976623535
PATH: /home/jovyan/work/data/240p/4sCVouieJUc.mp4
Converting <VideoCapture 0x7efe79a15f90> to numpy arrays
Captured 4.04 seconds (101 frames)
Elapsed time: 0.0668795108795166
Comparing ../../data/240p_watermark/4sCVouieJUc.mp4
../../data/240p_watermark/4sCVouieJUc.mp4 Comparison elapsed time: 0.7060105800628662
PATH: /home/jovyan/work/data/240p_watermark/4sCVouieJUc.mp4
Converting <VideoCapture 0x7efe79a04690> to numpy arrays
Captured 4.04 seconds (101 frames)
Elapsed time: 0.07160162925720215
Comparing ../../data/240p_vignette/4sCVouieJUc.mp4
../../data/240p_vignette/4sCVouieJUc.mp4

../../data/720p_black_and_white/bexPQO9gkSw.mp4 Comparison elapsed time: 0.8025686740875244
PATH: /home/jovyan/work/data/720p_black_and_white/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe79606990> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.27600574493408203
Comparing ../../data/720p_low_bitrate_4/bexPQO9gkSw.mp4
../../data/720p_low_bitrate_4/bexPQO9gkSw.mp4 Comparison elapsed time: 0.8327414989471436
PATH: /home/jovyan/work/data/720p_low_bitrate_4/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe7c4dedb0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.28336143493652344
Comparing ../../data/720p_watermark/bexPQO9gkSw.mp4
../../data/720p_watermark/bexPQO9gkSw.mp4 Comparison elapsed time: 0.7955114841461182
PATH: /home/jovyan/work/data/720p_watermark/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe79606990> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.31152963638305664
Comparing ../.

../../data/240p_flip_vertical/bexPQO9gkSw.mp4 Comparison elapsed time: 0.7590134143829346
PATH: /home/jovyan/work/data/240p_flip_vertical/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe79f80b10> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.07474660873413086
Comparing ../../data/240p_rotate_90_clockwise/bexPQO9gkSw.mp4
../../data/240p_rotate_90_clockwise/bexPQO9gkSw.mp4 Comparison elapsed time: 0.7742857933044434
PATH: /home/jovyan/work/data/240p_rotate_90_clockwise/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe7a0d38b0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.036405086517333984
Comparing ../../data/144p/bexPQO9gkSw.mp4
../../data/144p/bexPQO9gkSw.mp4 Comparison elapsed time: 0.7411658763885498
PATH: /home/jovyan/work/data/144p/bexPQO9gkSw.mp4
Converting <VideoCapture 0x7efe7c4dedb0> to numpy arrays
Captured 4.0344827586206895 seconds (117 frames)
Elapsed time: 0.0344390869140625
Comparing ../../data/144p_wate

../../data/480p_watermark/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.6552059650421143
PATH: /home/jovyan/work/data/480p_watermark/YRXsFtqOeAI.mp4
Converting <VideoCapture 0x7efe79f2e310> to numpy arrays
Captured 4.041666666666667 seconds (97 frames)
Elapsed time: 0.14330148696899414
Comparing ../../data/480p_vignette/YRXsFtqOeAI.mp4
../../data/480p_vignette/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.6335601806640625
PATH: /home/jovyan/work/data/480p_vignette/YRXsFtqOeAI.mp4
Converting <VideoCapture 0x7efe79f80070> to numpy arrays
Captured 4.041666666666667 seconds (97 frames)
Elapsed time: 0.1679549217224121
Comparing ../../data/480p_black_and_white/YRXsFtqOeAI.mp4
../../data/480p_black_and_white/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.6449966430664062
PATH: /home/jovyan/work/data/480p_black_and_white/YRXsFtqOeAI.mp4
Converting <VideoCapture 0x7efe79f5b1b0> to numpy arrays
Captured 4.041666666666667 seconds (97 frames)
Elapsed time: 0.13763189315795898
Comparing ../../data/480p_low_

../../data/144p_low_bitrate_4/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.70558762550354
PATH: /home/jovyan/work/data/144p_low_bitrate_4/YRXsFtqOeAI.mp4
Converting <VideoCapture 0x7efe79fcc890> to numpy arrays
Captured 4.041666666666667 seconds (97 frames)
Elapsed time: 0.015904664993286133
Comparing ../../data/144p_flip_vertical/YRXsFtqOeAI.mp4
../../data/144p_flip_vertical/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.6702346801757812
PATH: /home/jovyan/work/data/144p_flip_vertical/YRXsFtqOeAI.mp4
Converting <VideoCapture 0x7efe79f2e910> to numpy arrays
Captured 4.041666666666667 seconds (97 frames)
Elapsed time: 0.05304980278015137
Comparing ../../data/144p_rotate_90_clockwise/YRXsFtqOeAI.mp4
../../data/144p_rotate_90_clockwise/YRXsFtqOeAI.mp4 Comparison elapsed time: 0.8310985565185547
PATH: /home/jovyan/work/data/144p_rotate_90_clockwise/YRXsFtqOeAI.mp4
Elapsed time: 39.386152267456055
***************************
Processing asset 61 of 139: ../../data/1080p/H6Yizv1gwho.mp4
Converting 

## 3.- Extract aggregated metrics values to a pandas DataFrame

Once we have iterated through each and every asset of the dataset, it is time to drop the contents of the dictionary to a pandas DataFrame.
But before, other metrics computed by means of external scripts need to be collected (namely VMAF and MS-SSIM). Checkout Readme.md to see how to extract those metrics.

In [ ]:
dict_of_df = {k: pd.DataFrame(v) for k,v in metrics_dict.items()}
metrics_df = pd.concat(dict_of_df, axis=1).transpose().reset_index(inplace=False)

In [ ]:
metrics_path = '/home/jovyan/work/data-analysis/output'
real_path = os.path.realpath(metrics_path)
extra_metrics = ['vmaf', 'ms-ssim']

for index,row in metrics_df.iterrows():
    for metric in extra_metrics:

        asset_name = row['level_0'].split('/')[-1].split('.')[0]
        attack = row['level_1'].split('/')[3]
        dimension = attack.split('_')[0].replace('p','')
        attack_name = attack.replace('{}p'.format(dimension), dimension)
        log_path = '{}/{}/{}/{}/{}_{}.log'.format(metrics_path, metric, attack_name, asset_name, asset_name, dimension)

        print('LEVEL 0', row['level_0'])
        print('LEVEL 1:', row['level_1'])
        print('ASSET NAME:', asset_name)
        print('ATTACK:', attack)
        print('DIMENSION', dimension)
        print('ATTACK NAME', attack_name)
        print('PATH:', log_path)
        
        if os.path.isfile(log_path): 
            print('ADDING:',log_path)
            print('*****************************')
            metric_value = read_metric_log(log_path, metric)
            metrics_df.at[index, metric] = metric_value
        else:
            print('Path not found')



In [ ]:
metrics_df.head()

In [ ]:
metrics_df.to_csv('../output/metrics.csv')